### Import chats, 

In [21]:
import whatstk
from whatstk import df_from_txt_whatsapp

from deep_translator import GoogleTranslator

import emoji

from tqdm.notebook import tqdm_notebook
import time

In [2]:
chat = df_from_txt_whatsapp('../data/_chat.txt')

Deleting messages with links, images or audios

In [3]:
def delete_message_with(chat_dataframe, text_to_delete):
    """
        inputs: chat_dataframe: DataFrame, has to own a message column.
                text_to_delete: str. The text that if contained in the message column the row will be deleted.
        output: A DataFrame without the rows which message column contains the text_to_delete
    """
    return chat_dataframe[~chat_dataframe["message"].str.contains(text_to_delete, regex=False)]

In [4]:
chat = delete_message_with(chat, ".com/")
chat = delete_message_with(chat, "imagen omitida")
chat = delete_message_with(chat, "audio omitido")

In [5]:
chat["message"] = chat["message"].apply(lambda x: x.replace("\u200e", ""))

Create new column with string of the different emojis in the message

In [6]:
chat["emojis"] = chat["message"].apply(lambda message: ''.join(emoji.distinct_emoji_list(message)))

In [7]:
 def remove_emojis(row):
    """
        inputs: a row of a dataframe with a message column and a list of emojis column
        outputs: the row with the emojis removed
    """
    if row["emojis"] == "":
        return row["message"]
    else:
        replaced = row["message"]
        for emoji_ in row["emojis"]:
            replaced = replaced.replace(emoji_, "")
        return replaced

Removing the emojis from the message column

In [8]:
chat["message"] = chat.apply(remove_emojis, axis=1)

Create column with messages translated to english for performing NLP

In [9]:
def translate_text(message):
    """
        inputs: str. A message to be translated.
        outputs: str. The string translated to english
    """
    try:
        translation = GoogleTranslator(source='auto', target='en').translate(message)
        return translation
    except Exception as e:
        print(e)
        return ""

In [11]:
tqdm_notebook.pandas()
chat["message_translated"] = chat["message"].progress_apply(lambda message: translate_text(message))

  0%|          | 0/37760 [00:00<?, ?it/s]

11 --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated
8 --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated
11 --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated
24 --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated
("Connection broken: ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None)", ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))
500 --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated
19 --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated
2013 --> text must be a valid text with maximum 5000 character, otherwise it cannot be translated
328055730 --> text must be a valid text with maximum

In [14]:
chat.reset_index(inplace=True, drop=True)

In [20]:
chat.to_csv("../data/_chat_translated.csv", index=True)